# Deep Learning Question & Answer Chatbot

We will be implementing a chat bot that can answer questions given a set of sentences. The chatbot will use a subset of the Babi Data Set from Facebook Research; it already contains stories(sentences), queries(questions), and answers. 
Here is a link to the Babi Data Sets and the research paper this is based on:

Full Details: https://research.fb.com/downloads/babi/

- Jason Weston, Antoine Bordes, Sumit Chopra, Tomas Mikolov, Alexander M. Rush,
  "Towards AI-Complete Question Answering: A Set of Prerequisite Toy Tasks",
  http://arxiv.org/abs/1502.05698
  
The bot currently returns a yes or a no to each question asked. However, I plan on integrating a Natural Language Generation component to introduce some meaningful dialogue.  

## Imports

In [3]:
import pickle
import numpy as np

In [1]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [76]:
# For creating the model
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout
from keras.layers import add, dot, concatenate
from keras.layers import LSTM

## Load the Data

In [4]:
with open("train_qa.txt", "rb") as fp:   # Unpickling
    train_data =  pickle.load(fp) # List

In [5]:
with open("test_qa.txt", "rb") as fp:   # Unpickling
    test_data =  pickle.load(fp) # List

10:1 ratio for training data vs testing data; there are 10,000 points for train_data and 1,000 points for test_data. 

In [14]:
train_data[10]

(['Sandra',
  'went',
  'back',
  'to',
  'the',
  'hallway',
  '.',
  'Sandra',
  'moved',
  'to',
  'the',
  'office',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'office', '?'],
 'yes')

In [15]:
story_sentence = ' '.join(train_data[10][0]) #Story/Sentence
query_question = ' '.join(train_data[10][1]) #Query/Question
answer = train_data[10][2] #Answer to question

In [26]:
print("Sentence: ", story_sentence)
print("Question: ", query_question)
print("Answer:   ", answer)

Sentence:  Sandra went back to the hallway . Sandra moved to the office .
Question:  Is Sandra in the office ?
Answer:    yes


## Create a Vocabulary of all of the Words

In [19]:
# Set that contains the vocab words
vocab = set()

In [22]:
all_data = train_data + test_data 

for story, question , answer in all_data:
    # Creates a vocabulary of all the distinct words inside our dataset 
    vocab = vocab | set(story) # vocab ∪ Story. Continuously adds unique words
    vocab = vocab | set(question) # vocab ∪ question. Continuously adds unique words

In [23]:
# Add in the two possible answers 
vocab.add('no')
vocab.add('yes')

In [24]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [27]:
# Reserve 0 for Keras pad_sequences 
vocab_size = len(vocab) + 1 # + 1 to add an extra space for a 0 for Keras's pad_sequences

In [45]:
# Find longest story
longestStory = max(( (len(data[0])) for data in all_data )) 
longestQuery = max(( (len(data[1])) for data in all_data )) 

In [47]:
print(longestStory)
print(longestQuery)

156
6


## Vectorize Data

In [49]:
# Creates integer encoding for the sequences of words
tokenizer = Tokenizer(filters = [])
tokenizer.fit_on_texts(vocab) # This method creates the vocabulary index based on word frequency

In [52]:
tokenizer.word_index

{'journeyed': 1,
 'football': 2,
 'picked': 3,
 'daniel': 4,
 'bedroom': 5,
 'got': 6,
 'yes': 7,
 'took': 8,
 'milk': 9,
 'garden': 10,
 'down': 11,
 'went': 12,
 'travelled': 13,
 'kitchen': 14,
 'office': 15,
 'mary': 16,
 'back': 17,
 'moved': 18,
 'apple': 19,
 '.': 20,
 'left': 21,
 'put': 22,
 'grabbed': 23,
 'sandra': 24,
 'there': 25,
 'hallway': 26,
 'the': 27,
 'discarded': 28,
 'no': 29,
 'is': 30,
 'up': 31,
 'in': 32,
 'to': 33,
 'john': 34,
 'bathroom': 35,
 'dropped': 36,
 '?': 37}

In [55]:
trainStoryText = []
trainQueryText = []
trainAnswers = []

for story, query, answer in train_data:
    trainStoryText.append(story)
    trainQueryText.append(query)
    trainAnswers.append(answer)

In [59]:
 # Transforms each word in the sentences to a sequence of integers.
trainStorySeq = tokenizer.texts_to_sequences(trainStoryText)

In [71]:
def vectorizeStories(data, word_index = tokenizer.word_index, maxStoryLen = longestStory, maxQueryLen = longestQuery):
    """
    Vectorizes stories, queries, & answers into padded sequences. 
   
    Parameters: 
        data: All the data (Stories, Queries, Answers)
        word_index: A word index dictionary. Defaulted to our tokenizer.word_index
                    Can be overrided to other datasets or other sets of questions
        maxStoryLen: Length of the longest story (Will be used for the pad_sequences function)
        maxQueryLen: Length of the longest query (Will be used for the pad_sequences function)
        
        We need the max story & query length because we are using padded sequences; not every story/query 
        is the same length and our RNN that we're using for training needs everything to be the same length 
        We'll pad the inputs with 0s in case there's a story or query that is too short. Or we can cut down 
        a story or query if it is too long.
        
    Returns: 
        this (tuple): A tuple of the form (X, Q, A) (padded based on max lengths)    
    """
    X = [] # X := Stories
    Q = [] # Q := Queries
    A = [] # A := Answers (yes/no)
    
    for story, query, answer in data:
        # Convert the raw words into integers through a word index value
        
        # Grabs the word index for every word in story
        # [9, 34, ...]
        x = [word_index[word.lower()] for word in story]
        # Grabs the word index for every word in query
        q = [word_index[word.lower()] for word in query]
        
        # Index 0 is reserved since we are using pad sequences, so we add + 1
        a = np.zeros(len(word_index) + 1)
        
        # a is an empty matrix of NP zeros so we'll use numpy logic to create this assignment (Yes/No)
        a[word_index[answer]] = 1
        
        # We now append each set to their appropriate output list.
        X.append(x)
        Q.append(q)
        A.append(a)
    
    # Now that we have converted the words to numbers, we pad the sequences so they are all of equal length.
    X_padded_seqs = pad_sequences(X, maxlen = maxStoryLen)
    Q_padded_seqs = pad_sequences(Q, maxlen= maxQueryLen)
    answers = np.array(A)
    
    # Now that the sequences are padded based on their max length, the RNN can be trained on uniformly long sequences.
    # Returns tuple for unpacking. 
    return (X_padded_seqs, Q_padded_seqs, answers)

In [64]:
inputsTrain, queriesTrain, answersTrain = vectorizeStories(train_data)
inputsTest, queriesTest, answersTest = vectorizeStories(test_data)

In [70]:
print(inputsTrain)

[[ 0  0  0 ... 27  5 20]
 [ 0  0  0 ... 27 26 20]
 [ 0  0  0 ... 27 35 20]
 ...
 [ 0  0  0 ... 27  5 20]
 [ 0  0  0 ...  9 25 20]
 [ 0  0  0 ... 19 25 20]]


In [66]:
print(inputsTest)

[[ 0  0  0 ... 27  5 20]
 [ 0  0  0 ... 27 10 20]
 [ 0  0  0 ... 27 10 20]
 ...
 [ 0  0  0 ... 27 19 20]
 [ 0  0  0 ... 27 10 20]
 [ 0  0  0 ... 19 25 20]]


In [67]:
print(answersTest)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [73]:
tokenizer.word_index['yes']

7

In [74]:
tokenizer.word_index['no']

29

In [75]:
sum(answersTest)

array([  0.,   0.,   0.,   0.,   0.,   0.,   0., 497.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0., 503.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.])

We can see that we have 497 'yes's at index location 7 and 29 'no's at index location 29.

Our stories, queries, and answers are now successfully vectorized

## Create the model